In [4]:
import pandas as pd
import numpy as np

import csv

import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt

%matplotlib inline 
plt.rcParams['figure.figsize'] = [20,10]

import seaborn as sns
sns.set()
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split


In [5]:
const_dt = pd.read_csv('Construction_data.csv')

In [6]:
const_dt.head()

,GEO,Building_ID,School,project_type,Description,Phase,Status,Actual_start,Planned_end,Actual_end,Budget,Estimated_cost,Actual_cost,DSF,Actual_duration,planned_duration
0,1,M015,P.S. 15 - MANHATTAN,SCA CIP,FIRE ALARM,Scope,Complete,2013-07-24,2013-10-23,2013-11-12,16980,19165.0,19165.0,DSF: 0000739189,111,91.0
1,1,M015,P.S. 15 - MANHATTAN,SCA CIP,FIRE ALARM,Design,Complete,2013-11-20,2014-05-27,2014-04-11,67920,107963.0,107963.0,DSF: 0000739189,142,188.0
2,1,M015,P.S. 15 - MANHATTAN,SCA CIP,FIRE ALARM,Construction,Complete,2014-08-11,2015-08-10,2015-07-23,1273500,1219469.0,1129734.0,DSF: 0000739189,346,364.0
3,1,M015,P.S. 15 - MANHATTAN,SCA CIP,FIRE ALARM,"CM, F&E",Complete,2014-08-11,2015-08-10,2015-07-23,79240,170877.0,170877.0,DSF: 0000739189,346,364.0
4,1,M015,P.S. 15 - MANHATTAN,RCT CIP,ELECTRICAL LIGHTING FIXTURES,Scope,Complete,2014-10-30,2014-10-30,2014-10-30,19140,2111.0,2111.0,DSF: 0000798364,1,1.0


In [7]:
const_dt = const_dt.dropna(subset = ['Actual_cost'])
const_dt = const_dt.dropna(subset = ['planned_duration'])
const_dt = const_dt.dropna(subset = ['Actual_cost'])
const_dt = const_dt.dropna(subset = ['Estimated_cost'])
const_dt['Estimated_cost'] =pd.to_numeric(const_dt['Estimated_cost'], errors='coerce')

In [9]:
dummy_dt = pd.get_dummies(const_dt['Phase'])

In [10]:
st1 = pd.concat([const_dt['planned_duration'],const_dt['Estimated_cost'], dummy_dt], axis=1) 
st1.head()

,planned_duration,Estimated_cost,"CM, F&E","CM,Art,F&E",Construction,Design,F&E,Purch & Install,Scope
0,91.0,19165.0,0,0,0,0,0,0,1
1,188.0,107963.0,0,0,0,1,0,0,0
2,364.0,1219469.0,0,0,1,0,0,0,0
3,364.0,170877.0,1,0,0,0,0,0,0
4,1.0,2111.0,0,0,0,0,0,0,1


In [11]:
st2 = pd.concat([const_dt['School'], const_dt['planned_duration'],const_dt['Estimated_cost'], dummy_dt], axis=1) 
st2.head()

,School,planned_duration,Estimated_cost,"CM, F&E","CM,Art,F&E",Construction,Design,F&E,Purch & Install,Scope
0,P.S. 15 - MANHATTAN,91.0,19165.0,0,0,0,0,0,0,1
1,P.S. 15 - MANHATTAN,188.0,107963.0,0,0,0,1,0,0,0
2,P.S. 15 - MANHATTAN,364.0,1219469.0,0,0,1,0,0,0,0
3,P.S. 15 - MANHATTAN,364.0,170877.0,1,0,0,0,0,0,0
4,P.S. 15 - MANHATTAN,1.0,2111.0,0,0,0,0,0,0,1


In [12]:
st2[st2['School'].str.contains('P.S. 183 ROBERT')]

,School,planned_duration,Estimated_cost,"CM, F&E","CM,Art,F&E",Construction,Design,F&E,Purch & Install,Scope
133,P.S. 183 ROBERT L. STEVENSON,22.0,114840.0,0,0,0,0,0,1,0
134,P.S. 183 ROBERT L. STEVENSON,22.0,103948.0,0,0,0,0,0,1,0


In [13]:
y = const_dt['Actual_cost']
X_train, X_test, y_train, y_test = train_test_split(st1, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(454, 9) (454,)
(114, 9) (114,)


In [14]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_train)

In [17]:
coeff_df = pd.DataFrame(lm.coef_,st1.columns,columns=['Coefficient'])
coeff_df

,Coefficient
planned_duration,-3.669457e+02
Estimated_cost,9.668594e-01
"CM, F&E",5.995367e+04
"CM,Art,F&E",-4.408377e+04
Construction,-4.888600e+04
Design,-3.396467e+04
F&E,-1.767521e+06
Purch & Install,-2.115287e+04
Scope,-1.615883e+04


In [18]:
lm.predict(np.array(st1.loc[133,:]).reshape(1,-1))

array([135345.29203828])

In [19]:
X_train.loc[133,:]

planned_duration        22.0
Estimated_cost      114840.0
CM, F&E                  0.0
CM,Art,F&E               0.0
Construction             0.0
Design                   0.0
F&E                      0.0
Purch & Install          1.0
Scope                    0.0
Name: 133, dtype: float64